In [1]:
from FiinQuantX import FiinSession
from dotenv import load_dotenv
import os

# Load environment variables
load_dotenv()
username = os.getenv('USERNAME1')
password = os.getenv('PASSWORD1')

client = FiinSession(username=username, password=password).login()
fi = client.FiinIndicator()



In [ ]:
import pandas as pd
import time

# Lấy danh sách tickers từ FiinQuant (cách làm của bạn rất chính xác)
hose = list(client.TickerList(ticker="VNINDEX"))
hnx = list(client.TickerList(ticker="HNXINDEX"))
upcom = list(client.TickerList(ticker="UPCOMINDEX"))
tickers = list(set(hose + hnx + upcom))
print(f"Lấy được {len(tickers)} mã chứng khoán từ 3 sàn.")

# --- Tối ưu hóa việc fetch dữ liệu ---
# Chia danh sách tickers thành các lô nhỏ hơn (ví dụ: 100 mã mỗi lô)
def create_chunks(data, chunk_size):
    for i in range(0, len(data), chunk_size):
        yield data[i:i + chunk_size]

ticker_chunks = list(create_chunks(tickers, 100))
all_data = []

print(f"Chia thành {len(ticker_chunks)} lô để tải về...")

# Lặp qua từng lô và fetch dữ liệu
for i, chunk in enumerate(ticker_chunks):
    print(f"Đang xử lý lô {i+1}/{len(ticker_chunks)}...")
    try:
        chunk_data = client.Fetch_Trading_Data(
            realtime=False,
            tickers=chunk,
            fields=["open", "high", "low", "close", "volume"],
            adjusted=True,
            by="1d",
            period=1  # Lấy dữ liệu EOD của ngày gần nhất
        ).get_data()
        
        if chunk_data is not None and not chunk_data.empty:
            all_data.append(chunk_data)
        
        time.sleep(0.5) # Thêm một khoảng nghỉ nhỏ để tránh quá tải API
        
    except Exception as e:
        print(f"Lỗi khi xử lý lô {i+1}: {e}")

if all_data:
    final_eod_data = pd.concat(all_data, ignore_index=True)
    print("Tải dữ liệu EOD thành công!")
else:
    final_eod_data = pd.DataFrame()
    print("Không tải được dữ liệu nào.")

Lấy được 1605 mã chứng khoán từ 3 sàn.
Chia thành 17 lô để tải về...
Đang xử lý lô 1/17...
Fetching data, it may take a while. Please wait...


In [ ]:
# Hiển thị kết quả
display(final_eod_data.head())
display(final_eod_data.tail())
